In [9]:
import openai
from PIL import Image
import requests
import base64
import os


In [ ]:
client = openai.Client()


### Gerar Imagens

In [ ]:
nome="robo_praia"
model="dall-e-3"
prompt="Crie uma imagem com dois robôs em uma praia lendo livro de finanças"
size="1024x1024"
quality="hd"
style="vivid"


In [ ]:
resposta = client.images.generate(
    model=model,
    prompt=prompt,
    size=size,
    quality=quality,
    style=style,
    n=1
)


In [ ]:
resposta.data[0].revised_prompt


"Create an image featuring two robots sitting on a sandy beach, with gentle waves lapping at the shore nearby. Each of the robots is engrossed in reading a thick finance book. The cover of the book is labeled 'Finance'. The robots have a metallic sheen, reflecting the soft glow of the setting sun in the background. There are a few palm trees lining the edge of the beach, and a large seashell can be seen partially buried in the sand next to one of the robots."

In [ ]:
nome_arquivo = f"files/{nome}_{model}_{quality}_{style}.jpg"
image_url = resposta.data[0].url
image_data = requests.get(image_url).content
with open(nome_arquivo, "wb") as f:
    f.write(image_data)


### Image to Text

In [ ]:
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": "Descreva a imagem fornecida"},
            {"type": "image_url", "image_url": {
                "url": "https://harus.ind.br/wp-content/uploads/2023/02/5-praias-brasileiras.jpg"
            }}
        ]
    }]
)


In [ ]:
resposta.choices[0].message.contents


'A imagem retrata uma praia ao pôr do sol, com um céu colorido em tons de laranja, rosa e lilás. À esquerda, há palmeiras altas que se inclinam em direção à água. O chão é coberto de areia fina, e a linha de água reflete as cores do céu. Ao fundo, pode-se ver uma pequena colina e vegetação, com algumas estruturas visíveis, possivelmente de restaurantes ou bares à beira-mar. A cena transmite uma atmosfera tranquila e tropical, ideal para relaxamento.'

### Extraia placa de carro

In [7]:
texto = "Qual é a placa? Devolva apenas a placa!"


In [18]:
def encode_image(image_path):
    """Codifica a imagem em base64."""
    if not os.path.exists(image_path):
        print(f"Erro: Arquivo de imagem não encontrado em: {image_path}")
        return None
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


In [ ]:
# Caminho para a imagem (assumindo que está na pasta 'files' no diretório local)
image_path = os.path.join(os.getcwd(), "files", "placa.jpg")


In [ ]:
# Codifica a imagem para base64
base64_image = encode_image(image_path)


Erro: Arquivo de imagem não encontrado em: c:\Users\Diego\OneDrive\Documentos\Projetos\Python\IA\openAI\text_gen\src\samples\files\placa1.jpg


In [16]:
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": texto},
            {"type": "image_url", "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
                #"url": "https://quatrorodas.abril.com.br/wp-content/uploads/2018/09/imagem-e1536873662306.jpg?quality=70&strip=info&w=720&crop=1"
            }}
        ]
    }],
    max_tokens=1000
)


In [17]:
resposta.choices[0].message.content


'R103A19'

### OCR

In [ ]:
texto = "O que está escrito na imagem?"


In [ ]:
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": texto},
            {"type": "image_url", "image_url": {
                "url": "https://www.rbsdirect.com.br/imagesrc/17752154.jpg?w=700"
            }}
        ]
    }],
    max_tokens=1000
)


In [ ]:
resposta.choices[0].message.content


'O que está escrito na imagem é:\n\n"ISSO É UM PEDIDO:\n\nQue você seja muito feliz no seu grande dia. \n\nEste vestido tem mãos de uma brasileira, gera emprego e movimenta a economia local.\n\nFoi um prazer fazê-lo para você.\n\nEliana\nAteliê\nBrasileira 1/25"'

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What are in these images? Is there any difference between them?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)


In [ ]:
response.choices[0].message.content


'Both images depict the same scene of a natural landscape featuring a wooden pathway or boardwalk that runs through tall grass, with grassy fields and trees in the background under a blue sky adorned with clouds.\n\nSince the images appear identical, there are no discernible differences between them.'